# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

In [20]:
class Wcode < Hash
  def initialize(word = Group::Identity)
    @word = word
    Rseq.each{|x| self.store(x.char, [])}
    if word != Group::Identity
      word.flatten.factors.each_with_index do |c, i|
        k = i
        self[c.char].insert(-1, 2*k)
        self[c.inverse.char].insert(0,2*k+1)
      end
    end
  end
  attr_reader :word
  
  def show()
    self.values.map(&:to_s).join
  end
  def to_a()
    self.values.flatten
  end
  def size()
    self.to_a.size
  end
  #---
  def four_tuple(odd_nums)
    arr = [odd_nums[0], odd_nums[0]+1, odd_nums[1], (odd_nums[1]+1)%self.size]
    FourTuple.new(self.to_a.select{|n| arr.include?(n)}, self.size)
  end
  #---
  def devide(odd_nums)
    facs = []
    facs[0] = self.map{|key, arr| [key, arr.select{|x| x <= odd_nums[0] or odd_nums[1] < x}]}.to_h
    facs[1] = self.map{|key, arr| [key, arr.select{|x| odd_nums[0] < x and x <= odd_nums[1]}]}.to_h
    facs.map! do |f|
      wc = self.class.new
      f.each{|k,a| wc.store(k, a)}
      wc
    end
    return facs
  end
  #---
end; nil

class FourTuple < Array
  def initialize(arr, mod)
    @modulo = mod
    4.times{|i| self[i] = arr[i]}
    @odd_nums = self.select{|n| n.odd?}.sort
  end
  
  def linking?
    odd1 = @odd_nums[0]
    (self.index(odd1) - self.index(odd1+1) + 1).odd? 
  end
  def sign
    odd1 = @odd_nums[0]
    idx = self.index(odd1)
    sign0 = (idx < self.index(odd1+1)) ? 1 : -1
    sign2 = self[idx+1].even? ? 1 : -1
    sign = sign0*sign2
  end
end

:sign

In [22]:
myw = comms[1]
wcode = Wcode.new(myw)
p wcode.devide([1,3]).map(&:show)
nil

["[5, 0][6][1, 4][7][][][][][][][][]", "[][3][][2][][][][][][][][]"]


In [7]:
#myw = Word.new(a[1], b[2].inverse)
#myw = Word.new(a[1])^3
c = Word.new(a[1], b[1], a[1].inverse); myw = c
myw = comms[1]
#myw = Word.new(a[1])*(comms[2]^2)
#myw = Word.new(a[1])*comms[2]*c*comms[2]*(c.inverse)
myw.show

"abAB"

In [8]:
wcode = Wcode.new(myw)
p myw.show, wcode.show

odd_pairs=(1..wcode.size/2).to_a.map{|x| 2*x-1}.combination(2)

#--------------------------------------
coprods = odd_pairs.map do |odds|
  order = wcode.four_tuple(odds)
  {order: order, linking: order.linking?, sign: order.sign} #, factors: factors, prewords: prewords, words: words}
end
#--------------------------------------

#--------------------------------------
coprods.select{|h| h[:linking]}.each do |h|
  p "#{h[:order]}"
#  p "#{h[:linking]}"
  p "#{h[:sign]}"
# strings = h[:factors].map{|f| f.values.map(&:to_s).join}
#  p strings.join(" , ")
#  p h[:prewords].join(" , ")
#  p ((h[:sign]==1) ? "+" : "-") + h[:words].join("(x)")
  printf(".....\n")
end
#--------------------------------------
nil

Interrupt: 

In [9]:
  
   # contract fac1 & fac2
  #
  # shift numbers in fac1 & fac2
  factors = [fac1, fac2].map do |f|
    sorted = f.values.flatten.sort
    f.map{|key, arr| [key, arr.map{|v| sorted.index(v)}]}.to_h
  end
  # code --> word
  prewords = [fac1, fac2].map do |f|
    chars = (0..size/2).to_a.map{|i| f.map{|key, arr| key if arr.include?(2*i)}.join}
     Word.new(chars.join)
  end
  words = prewords.map do |ww|
    ww = ww.dup.contract 
    wcp = Group::Identity
    while (wcp.size < ww.size)
      ww.factors = wcp.factors unless wcp == Group::Identity
      wcp = ww.cyclic_permutation.contract
    end
    ww
  end
  {pair: pair, order: order, parity: parity, sign: sign, factors: factors, prewords: prewords, words: words}
end
printf("----------------------\n")



SyntaxError: <main>:23: syntax error, unexpected keyword_end, expecting end-of-input

### TODO
$[a_{1},b_{1}]$ の処理において、
* [ ] [5, 0, 6, 7] の sign が正しくないようだ。
* [ ] a(x)A の集計もおかしい。

In [10]:
'''
printf("----------------------\n")
cprds = coprods.delete_if{|prd| prd[:parity] == "even"}

i = 0
while cprds.size > 0
  collection = [cprds.pop]
  collection += cprds.select{|other| other[:words] == collection[0][:words]}
#  p collection.map{|h| ((h[:sign]==1) ? "+" : "-") + "#{h[:words].map(&:show).join("(x)")}" }.join(" ")
  coeff = collection.map{|h| h[:sign]}.sum
  term = collection[0][:words].map(&:show).join("(x)")
  p case coeff 
    when 0 then "0"
    when 1 then term
    else coeff.to_s + "*" + term
  end

  cprds = cprds.delete_if{|h| h[:words] == collection[0][:words]}
end 
'''

"\nprintf(\"----------------------\\n\")\ncprds = coprods.delete_if{|prd| prd[:parity] == \"even\"}\n\ni = 0\nwhile cprds.size > 0\n  collection = [cprds.pop]\n  collection += cprds.select{|other| other[:words] == collection[0][:words]}\n#  p collection.map{|h| ((h[:sign]==1) ? \"+\" : \"-\") + \"\#{h[:words].map(&:show).join(\"(x)\")}\" }.join(\" \")\n  coeff = collection.map{|h| h[:sign]}.sum\n  term = collection[0][:words].map(&:show).join(\"(x)\")\n  p case coeff \n    when 0 then \"0\"\n    when 1 then term\n    else coeff.to_s + \"*\" + term\n  end\n\n  cprds = cprds.delete_if{|h| h[:words] == collection[0][:words]}\nend \n"